## ■ (10기 최종 포트폴리오) 정상폐/질병폐 사진분류 인공신경망 홈페이지(게시글 87번)

yys31 폴더를 d드라이브 밑에 둔다.

### ■ 오버피팅 안생기게 정확도를 올릴려면?

1. 사이즈를 64 x 64
2. vgg로 구현
3. 사진 사이즈가 크면 클수록 학습이 잘되긴 하지만 메모리가 터지는 문제가 발생할 수 있음.

### ■ R에서 폐사진 분류하는 코드.txt

### 현재 이코드는 홈페이지 올리기전에 코드로써 이코드로 ui.R과 server.R로 나누기 전임.

In [ ]:
setwd("D:\\lungs\\yys31")

packages <- c('imager', "shiny", "jpeg", "png", "reticulate", "devtools")

if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

if (length(setdiff("keras", rownames(installed.packages()))) > 0) {
  devtools::install_github("rstudio/keras") 
}

require(imager)
require(shiny)
require(jpeg)
require(png)
library(reticulate)
library(keras)

#setwd(tempfile())      
#setwd("/Users/aiden/Desktop/data/cifar10_densenet")

load("envir.RData")
model<<-load_model_hdf5("lung.h5")

synsets <<- readLines("synset.txt")

server <- shinyServer(function(input, output) {
  ntext <- eventReactive(input$goButton, {
    print(input$url)
    if (input$url == "http://") {
      NULL
    } else {
      tmp_file <- tempfile()
      download.file(input$url, destfile = tmp_file, mode = 'wb')
      tmp_file
    }
  })
  
  output$originImage = renderImage({
    list(src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'sample.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'sample.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    },
    title = "Original Image")
  }, deleteFile = FALSE)
  
  output$res <- renderText({
    src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'sample.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'sample.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    }
    
    img <- load.image(src)
    plot(img)
    img <- image_load(src, target_size = c(32,32))
    img
    x <- image_to_array(img)
    # ensure we have a 4d tensor with single element in the batch dimension,
    x <- array_reshape(x, c(1, dim(x)))
    
    # normalize
    x[,,,1] <- x[,,,1] /255.0
    x[,,,2] <- x[,,,2] /255.0
    x[,,,3] <- x[,,,3] /255.0
    
    # predcit
    preds <- model %>% predict(x)
    
    # output result as string
    
    order(preds[1,], decreasing = TRUE)
    max.idx <- order(preds[1,], decreasing = TRUE)[1]
    max.idx
    result <- synsets[max.idx]
    res_str <- ""
    tmp <- strsplit(result[1], " ")[[1]]
    res_str <- paste0(res_str, tmp[2])
    res_str
  })
})




require(imager)
require(shiny)
require(jpeg)
require(png)

ui <- shinyUI(
  fluidPage(
    includeCSS("bootstrap.css"),
    
    pageWithSidebar(
      headerPanel(title = 'Lung classificaiton',
                  windowTitle = 'Lung Classification'),
      
      fluidRow(
        column(1),
        column(9,
               tabsetPanel(
                 id = "tabs",
                 tabPanel("Upload Image",
                          fileInput('file1', 'Upload a PNG / JPEG File:')),
                 tabPanel(
                   "Use the URL",
                   textInput("url", "Image URL:", "http://"),
                   actionButton("goButton", "Go!")
                 )
               ),
               h3(titlePanel("DESCRIPTION - Image Classification")),
               h3(titlePanel("Image Classification"))
               
        ),
        column(2)
      ),
      
      
      mainPanel(
        h3("Image"),
        tags$hr(),
        imageOutput("originImage", height = "auto"),
        tags$hr(),
        h3("What is this?"),
        tags$hr(),
        verbatimTextOutput("res")
      )
      
      
    )))

shinyApp(ui = ui, server = server)

### ■ 이미지 사이즈 변경하는 코드

In [ ]:
import  cv2

import  os 

import  numpy  as np




path = "D:\\data\\lung\\image" # 경로는 자기 경로에 맞게 지정해주기!




file_list = os.listdir(path)




for k in file_list:




    img = cv2.imread(path + '\\' + k)

    width, height = img.shape[:2]

    resize_img = cv2.resize(img, (32 , 32), interpolation=cv2.INTER_CUBIC)

    cv2.imwrite( "D:\\data\\lung\\image\\resize\\" + k, resize_img)


###  ※ setwd부분은 경로설정하는 부분인데 자기경로에 맞춰서 수정해주면됨.

### 문제. 위의 코드로 ui.R과 server.R을 생성해서 홈페이지를 구현하시오!

### ■ ui.R

In [ ]:
shinyUI(
  fluidPage(
    includeCSS("bootstrap.css"),
    
    pageWithSidebar(
      headerPanel(title = 'Lung classificaiton',
                  windowTitle = 'Lung Classification'),
      
      fluidRow(
        column(1),
        column(9,
               tabsetPanel(
                 id = "tabs",
                 tabPanel("Upload Image",
                          fileInput('file1', 'Upload a PNG / JPEG File:')),
                 tabPanel(
                   "Use the URL",
                   textInput("url", "Image URL:", "http://"),
                   actionButton("goButton", "Go!")
                 )
               ),
               h3(titlePanel("DESCRIPTION - Image Classification")),
               h3(titlePanel("Image Classification"))
               
        ),
        column(2)
      ),
      
      
      mainPanel(
        h3("Image"),
        tags$hr(),
        imageOutput("originImage", height = "auto"),
        tags$hr(),
        h3("What is this?"),
        tags$hr(),
        verbatimTextOutput("res")
      )
      
      
    )))

### ■ server.R

In [ ]:
packages <- c('imager', "shiny", "jpeg", "png", "reticulate", "devtools")

if (length(setdiff(packages, rownames(installed.packages()))) > 0) {
  install.packages(setdiff(packages, rownames(installed.packages())))  
}

if (length(setdiff("keras", rownames(installed.packages()))) > 0) {
  devtools::install_github("rstudio/keras") 
}

require(imager)
require(shiny)
require(jpeg)
require(png)
library(reticulate)
library(keras)

#setwd(tempfile())      
#setwd("/Users/aiden/Desktop/data/cifar10_densenet")

load("envir.RData")
model<<-load_model_hdf5("lung.h5")

synsets <<- readLines("synset.txt")

shinyServer(function(input, output) {
  ntext <- eventReactive(input$goButton, {
    print(input$url)
    if (input$url == "http://") {
      NULL
    } else {
      tmp_file <- tempfile()
      download.file(input$url, destfile = tmp_file, mode = 'wb')
      tmp_file
    }
  })
  
  output$originImage = renderImage({
    list(src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'sample.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'sample.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    },
    title = "Original Image")
  }, deleteFile = FALSE)
  
  output$res <- renderText({
    src = if (input$tabs == "Upload Image") {
      if (is.null(input$file1)) {
        if (input$goButton == 0 || is.null(ntext())) {
          'sample.jpg'
        } else {
          ntext()
        }
      } else {
        input$file1$datapath
      }
    } else {
      if (input$goButton == 0 || is.null(ntext())) {
        if (is.null(input$file1)) {
          'sample.jpg'
        } else {
          input$file1$datapath
        }
      } else {
        ntext()
      }
    }
    
    img <- load.image(src)
    plot(img)
    img <- image_load(src, target_size = c(32,32))
    img
    x <- image_to_array(img)
    # ensure we have a 4d tensor with single element in the batch dimension,
    x <- array_reshape(x, c(1, dim(x)))
    
    # normalize
    x[,,,1] <- x[,,,1] /255.0
    x[,,,2] <- x[,,,2] /255.0
    x[,,,3] <- x[,,,3] /255.0
    
    # predcit
    preds <- model %>% predict(x)
    
    # output result as string
    
    order(preds[1,], decreasing = TRUE)
    max.idx <- order(preds[1,], decreasing = TRUE)[1]
    max.idx
    result <- synsets[max.idx]
    res_str <- ""
    tmp <- strsplit(result[1], " ")[[1]]
    res_str <- paste0(res_str, tmp[2])
    res_str
  })
})


require(imager)
require(shiny)
require(jpeg)
require(png)


### ■ 홈페이지 만드는 R 코드

In [ ]:

library(rsconnect)



rsconnect::setAccountInfo(name='lullaby0419', 
                          token='76A6ABEE4913A7103F13E9CB0DBD604D', 
                          secret='보안')





rsconnect::deployApp('d:\\lungs\\yys31',appName = "mnist")

### ■ 테스트 할 내용. 오버피팅 안생기게 정확도를 올리겠금 다시 모델을 생성해보시오!

 https://www.nature.com/articles/s41598-019-42557-4/figures/1  
 https://arxiv.org/ftp/arxiv/papers/1807/1807.03058.pdf